<a href="https://colab.research.google.com/github/PrasanthV454/For_Silver/blob/master/ResNet_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
from keras.layers import *
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from IPython.display import Image

batch_size = 48
width=150
height=150
epochs=20
NUM_TRAIN=2000
NUM_TEST=1000
dropout_rate=0.2
input_shape=(height,width,3)

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
conv_base=ResNet50(weights='imagenet',include_top=False,input_shape=input_shape)

In [3]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

--2020-04-01 05:27:07--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.193.24.126, 2600:1417:76:586::e59, 2600:1417:76:58e::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.193.24.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip’

kagglecatsanddogs_3 100%[===================>] 786.68M   215MB/s    in 3.8s    

2020-04-01 05:27:12 (209 MB/s) - ‘kagglecatsanddogs_3367a.zip’ saved [824894548/824894548]



In [0]:
!unzip -qq kagglecatsanddogs_3367a.zip -d dog_vs_cat

In [5]:
!ls dog_vs_cat/PetImages/

Cat  Dog


In [6]:
original_dataset_dir='./dog_vs_cat/PetImages'
cat_images = glob.glob(os.path.join(original_dataset_dir, "Cat", '*.jpg'))
dog_images = glob.glob(os.path.join(original_dataset_dir, "Dog", '*.jpg'))
print(len(cat_images))
print(len(dog_images))

12500
12500


In [0]:
base_dir = './data/dog_vs_cat_small'
os.makedirs(base_dir, exist_ok=True)
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)
test_dir = os.path.join(base_dir, 'test')
os.makedirs(test_dir, exist_ok=True)
train_cats_dir = os.path.join(train_dir, 'cats')
os.makedirs(train_cats_dir, exist_ok=True)
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.makedirs(train_dogs_dir, exist_ok=True)
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.makedirs(validation_cats_dir, exist_ok=True)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.makedirs(validation_dogs_dir, exist_ok=True)
test_cats_dir = os.path.join(test_dir, 'cats')
os.makedirs(test_cats_dir, exist_ok=True)
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.makedirs(test_dogs_dir, exist_ok=True)

fnames = cat_images[:NUM_TRAIN//2]
for fname in fnames:
    dst = os.path.join(train_cats_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

offset = NUM_TRAIN//2

fnames = cat_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(validation_cats_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

offset = offset + NUM_TEST // 2

fnames = cat_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(test_cats_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

fnames = dog_images[:NUM_TRAIN//2]
for fname in fnames:
    dst = os.path.join(train_dogs_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)


offset = NUM_TRAIN//2

fnames = dog_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(validation_dogs_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

offset = offset + NUM_TEST // 2

fnames = dog_images[offset:offset + NUM_TEST // 2]
for fname in fnames:
    dst = os.path.join(test_dogs_dir, os.path.basename(fname))
    shutil.copyfile(fname, dst)

In [8]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500
total test cat images: 500
total test dog images: 500


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [10]:
#model =  models.Sequential()
#model.add(conv_base)
#model.add(layers.GlobalMaxPooling2D(name="gap"))
#model.add(layers.Dense(2, activation='softmax', name="fc_out"))

model =  Sequential([
                     conv_base,
                     GlobalAveragePooling2D(name="gap"),
                     Dense(2, activation='softmax', name="fc_out")
])

In [11]:
conv_base.trainable = False
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 5, 5, 2048)        23587712  
_________________________________________________________________
gap (GlobalAveragePooling2D) (None, 2048)              0         
_________________________________________________________________
fc_out (Dense)               (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [12]:
from keras import optimizers
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history= model.fit_generator(
    train_generator,
    steps_per_epoch=NUM_TRAIN//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1,
    use_multiprocessing=True,
    workers=4)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
41/41 [==============================] - 378s 9s/step - loss: 0.8258 - acc: 0.5185 - val_loss: 0.7847 - val_acc: 0.5000
Epoch 2/20
41/41 [==============================] - 373s 9s/step - loss: 0.7356 - acc: 0.5737 - val_loss: 0.7906 - val_acc: 0.5000
Epoch 3/20
41/41 [==============================] - 375s 9s/step - loss: 0.7049 - acc: 0.5940 - val_loss: 0.7957 - val_acc: 0.5000
Epoch 4/20
41/41 [==============================] - 373s 9s/step - loss: 0.6561 - acc: 0.6387 - val_loss: 0.8018 - val_acc: 0.5000
Epoch 5/20
41/41 [==============================] - 373s 9s/step - loss: 0.6082 - acc: 0.6763 - val_loss: 0.8098 - val_acc: 0.5000
Epoch 6/20
41/41 [==============================] - 374s 9s/step - loss: 0.5928 - acc: 0.6837 - val_loss: 0.8140 - val_acc: 0.5000
Epoch 7/20
41/41 [==============================] - 373s 9s/step - loss: 0.5524 - acc: 0.7297 - val_loss: 0.8192 - v

In [13]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

result_loss,result_acc=model.evaluate(test_generator)
print(result_acc)

Found 1000 images belonging to 2 classes.
21/21 [==============================] - 135s 6s/step
0.5000000007152557
